In [ ]:
# Transformer 다국어 기계 번역 모델 분류기 
# - Hugging Face 라이브러리 적용
# - AI HUB 금융 학술논문/공시정보/뉴스/규정/보고서 다국어 번역 데이터셋 적용
# - 입력된 문장을 다국어 기계 번역 모델 분류기을 통한 학술논문(0)/공시정보(1)/뉴스(2)/규정(3)/보고서(4) 분류
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 금융 학술논문/공시정보/뉴스/규정/보고서 다국어 번역 데이터셋 전처리
# - 병렬 문장쌍 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [2]:
import torch
import tqdm as notebook_tqdm
import numpy as np
import glob, json, re, os, random, csv, zipfile

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.8.0+cu129 cuda
CUDA 사용 가능 여부: True
PyTorch CUDA 버전: 12.9
빌드 정보: 2.8.0+cu129
사용 중인 GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [18]:
# AI Hub 금융 학술 논문 데이터셋(train_ko.txt, train_en.txt) -> CSV로 변환해 파인 튜닝
# 원본 데이터는 train_ko.txt, train_en.txt로 분리 -> 병렬 문장쌍을 만들어야 함
# 파인튜닝시 양방향 번역을 지원하려면 같은 문장쌍은 en->ko, ko->en 두방향으로 모두 포함해야 함
# CSV 구조 예시
# - src,tgt,src_lang,tgt_lang,label
# - You can buy it from a convenience store try it out.,편의점에서 사실 수 있으니 시도해보시길 바랍니다.,en,ko
# - 편의점에서 사실 수 있으니 시도해보시길 바랍니다.,You can buy it from a convenience store try it out.,ko,en
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# AI Hub 금융 학술 논문 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_article_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_article_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines_article = f_en.read().splitlines()
    ko_lines_article = f_ko.read().splitlines()

limit = 1000 # 데이터 개수 제한 (예: 100개)
en_lines_article = en_lines_article[:limit]
ko_lines_article = ko_lines_article[:limit]

split_idx = int(len(en_lines_article) * 0.9) # train/valid split(90 : 10)
train_en_article, valid_en_article = en_lines_article[:split_idx], en_lines_article[split_idx:]
train_ko_article, valid_ko_article = ko_lines_article[:split_idx], ko_lines_article[split_idx:]
print('금융 학술 논문 : ', len(train_en_article), len(valid_en_article))

# AI Hub 금융 공시 정보 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_disclosure_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_disclosure_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines_disclosure = f_en.read().splitlines()
    ko_lines_disclosure = f_ko.read().splitlines()

limit = 1000 # 데이터 개수 제한 (예: 100개)
en_lines_disclosure = en_lines_disclosure[:limit]
ko_lines_disclosure = ko_lines_disclosure[:limit]

split_idx = int(len(en_lines_article) * 0.9) # train/valid split(90 : 10)
train_en_disclosure, valid_en_disclosure = en_lines_disclosure[:split_idx], en_lines_disclosure[split_idx:]
train_ko_disclosure, valid_ko_disclosure = ko_lines_disclosure[:split_idx], ko_lines_disclosure[split_idx:]
print('금융 공시 정보 : ', len(train_en_disclosure), len(valid_en_disclosure))

# AI Hub 금융 뉴스 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_news_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_news_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines_news = f_en.read().splitlines()
    ko_lines_news = f_ko.read().splitlines()

limit = 1000 # 데이터 개수 제한 (예: 100개)
en_lines_news = en_lines_news[:limit]
ko_lines_news = ko_lines_news[:limit]

split_idx = int(len(en_lines_article) * 0.9) # train/valid split(90 : 10)
train_en_news, valid_en_news = en_lines_news[:split_idx], en_lines_news[split_idx:]
train_ko_news, valid_ko_news = ko_lines_news[:split_idx], ko_lines_news[split_idx:]
print('금융 뉴스 : ', len(train_en_news), len(valid_en_news))

# AI Hub 금융 규정 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_regulation_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_regulation_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines_regulation = f_en.read().splitlines()
    ko_lines_regulation = f_ko.read().splitlines()

limit = 1000 # 데이터 개수 제한 (예: 100개)
en_lines_regulation = en_lines_regulation[:limit]
ko_lines_regulation = ko_lines_regulation[:limit]

split_idx = int(len(en_lines_article) * 0.9) # train/valid split(90 : 10)
train_en_regulation, valid_en_regulation = en_lines_regulation[:split_idx], en_lines_regulation[split_idx:]
train_ko_regulation, valid_ko_regulation = ko_lines_regulation[:split_idx], ko_lines_regulation[split_idx:]
print('금융 규정 : ', len(train_en_regulation), len(valid_en_regulation))

# AI Hub 금융 보고서 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_report_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_report_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines_report = f_en.read().splitlines()
    ko_lines_report = f_ko.read().splitlines()

limit = 1000 # 데이터 개수 제한 (예: 100개)
en_lines_report = en_lines_report[:limit]
ko_lines_report = ko_lines_report[:limit]

split_idx = int(len(en_lines_article) * 0.9) # train/valid split(90 : 10)
train_en_report, valid_en_report = en_lines_report[:split_idx], en_lines_report[split_idx:]
train_ko_report, valid_ko_report = ko_lines_report[:split_idx], ko_lines_report[split_idx:]
print('금융 보고서 : ', len(train_en_report), len(valid_en_report))

금융 학술 논문 :  900 100
금융 공시 정보 :  900 100
금융 뉴스 :  900 100
금융 규정 :  900 100
금융 보고서 :  900 100


In [19]:
# 병렬 데이터 생성 - 편향되지 않은 데이터셋 생성
# 영어->한국어, 한국어->영어, 
# label -> 학술논문(0)/공시정보(1)/뉴스(2)/규정(3)/보고서(4) 분류

# 출력 디렉토리
out_dir = './llm_data/ai_hub_finance_classification'
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
    print(f'폴더 생성 완료: {out_dir}')
else:
    print(f'이미 존재하는 폴더: {out_dir}')    

# 문서 유형별 라벨 정의
label_map = {
    'article': 0,       # 금융 학술 논문
    'disclosure': 1,    # 금융 공시 정보
    'news': 2,          # 금융 뉴스
    'regulation': 3,    # 금융 규정
    'report': 4         # 금융 보고서
}

# 데이터셋 생성 함수
def create_csv(train_en, train_ko, valid_en, valid_ko, doc_type):
    num_labels = label_map[doc_type]
    # train
    train_file = os.path.join(out_dir, f'train_{doc_type}.csv')
    with open(train_file, 'w', encoding='utf-8', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang', 'label'])
        for en, ko in zip(train_en, train_ko):
            if not en or not ko:
                continue
            # 영어 -> 한국어
            writer.writerow([en, ko, 'en', 'ko', num_labels])
            # 한국어 -> 영어
            writer.writerow([ko, en, 'ko', 'en', num_labels])
    
    # valid
    valid_file = os.path.join(out_dir, f'valid_{doc_type}.csv')
    with open(valid_file, 'w', encoding='utf-8', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang', 'label'])
        for en, ko in zip(valid_en, valid_ko):
            if not en or not ko:
                continue
            # 영어 -> 한국어
            writer.writerow([en, ko, 'en', 'ko', num_labels])
            # 한국어 -> 영어
            writer.writerow([ko, en, 'ko', 'en', num_labels])
    
    print(f"{doc_type} 데이터셋 저장 완료: train/valid CSV")

# 데이터셋 생성 호출
create_csv( # 금융 학술 논문 데이터셋 생성
    train_en=train_en_article, 
    train_ko=train_ko_article,
    valid_en=valid_en_article, 
    valid_ko=valid_ko_article,
    doc_type='article'
)
create_csv( # 금융 공시 정보 데이터셋 생성
    train_en=train_en_disclosure, 
    train_ko=train_ko_disclosure,
    valid_en=valid_en_disclosure, 
    valid_ko=valid_ko_disclosure,
    doc_type='disclosure'
)
create_csv( # 금융 뉴스 데이터셋 생성
    train_en=train_en_news, 
    train_ko=train_ko_news,
    valid_en=valid_en_news, 
    valid_ko=valid_ko_news,
    doc_type='news'
)
create_csv( # 금융 규정 데이터셋 생성
    train_en=train_en_regulation, 
    train_ko=train_ko_regulation,
    valid_en=valid_en_regulation, 
    valid_ko=valid_ko_regulation,
    doc_type='regulation'
)
create_csv( # 금융 보고서 데이터셋 생성
    train_en=train_en_report, 
    train_ko=train_ko_report,
    valid_en=valid_en_report, 
    valid_ko=valid_ko_report,
    doc_type='report'
)

이미 존재하는 폴더: ./llm_data/ai_hub_finance_classification
article 데이터셋 저장 완료: train/valid CSV
disclosure 데이터셋 저장 완료: train/valid CSV
news 데이터셋 저장 완료: train/valid CSV
regulation 데이터셋 저장 완료: train/valid CSV
report 데이터셋 저장 완료: train/valid CSV


In [22]:
# 데이터 전처리 - csv 파일 통합

# 출력 디렉토리
out_dir = './llm_data/ai_hub_finance_classification'

# 통합 파일 경로
train_all_file = os.path.join(out_dir, 'train_all.csv')
valid_all_file = os.path.join(out_dir, 'valid_all.csv')

# 공통 헤더
header = ['src', 'tgt', 'src_lang', 'tgt_lang', 'label']

# train_all.csv 생성
with open(train_all_file, 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(header)

    # train_*.csv 파일들 읽어서 합치기
    for file in glob.glob(os.path.join(out_dir, 'train_*.csv')):
        with open(file, 'r', encoding='utf-8') as f_in:
            reader = csv.reader(f_in)
            for i, row in enumerate(reader):
                if i == 0:
                    continue
                if row:
                    writer.writerow(row)
print(f"통합 학습 데이터셋 생성 완료: {train_all_file}")

# valid_all.csv 생성
with open(valid_all_file, 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(header)

    # valid_*.csv 파일들 읽어서 합치기
    for file in glob.glob(os.path.join(out_dir, 'valid_*.csv')):
        with open(file, 'r', encoding='utf-8') as f_in:
            reader = csv.reader(f_in)
            for i, row in enumerate(reader):
                if i == 0:
                    continue
                if row:
                    writer.writerow(row)
print(f"통합 검증 데이터셋 생성 완료: {valid_all_file}")


통합 학습 데이터셋 생성 완료: ./llm_data/ai_hub_finance_classification\train_all.csv
통합 검증 데이터셋 생성 완료: ./llm_data/ai_hub_finance_classification\valid_all.csv


In [23]:
# Hugging Face Dataset 로딩
from datasets import load_dataset

# csv 파일 로딩
train_dataset = load_dataset('csv', data_files={'train': './llm_data/ai_hub_finance_classification/train_all.csv'})['train']
valid_dataset = load_dataset('csv', data_files={'valid': './llm_data/ai_hub_finance_classification/valid_all.csv'})['valid']

print(train_dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

{'src': 'From 2019, it was mandatory for companies with total assets of KRW 2 trillion or more as of the end of the previous year, and from 2020, the mandatory target was expanded to companies with total assets of KRW 500 billion or more as of the end of the previous year.', 'tgt': '2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.', 'src_lang': 'en', 'tgt_lang': 'ko', 'label': 0}


In [ ]:
# 토큰화 및 데이터셋 생성
from transformers import AutoTokenizer

# 토큰 모델 로드 및 객체 생성
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# 토큰화 함수
def tokenize_function(example):
    return tokenizer(example['src'], truncation=True, padding='max_length', max_length=256)

# 데이터셋 생성
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AI\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
# 모델 정의
from transformers import AutoModelForSequenceClassification

# 모델 로드 및 객체 생성
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=5 # 학술논문, 공시정보, 뉴스, 규정, 보고서
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from transformers import TrainingArguments, Trainer

# 학습 설정
training_args = TrainingArguments(
    output_dir="./llm_models/results_finance_classification",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)


C:\Users\AI\AppData\Local\Temp\ipykernel_29384\253437258.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
# 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.492500,0.476776
2,0.177600,0.518957
3,0.089800,0.515019


TrainOutput(global_step=1689, training_loss=0.23191845522892413, metrics={'train_runtime': 4191.0493, 'train_samples_per_second': 6.442, 'train_steps_per_second': 0.403, 'total_flos': 3552094923264000.0, 'train_loss': 0.23191845522892413, 'epoch': 3.0})

In [28]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/translation_model_finance_classification")
tokenizer.save_pretrained("./llm_models/translation_model_finance_classification")

('./llm_models/translation_model_finance_classification\\tokenizer_config.json',
 './llm_models/translation_model_finance_classification\\special_tokens_map.json',
 './llm_models/translation_model_finance_classification\\vocab.txt',
 './llm_models/translation_model_finance_classification\\added_tokens.json',
 './llm_models/translation_model_finance_classification\\tokenizer.json')

In [30]:
# 평가
trainer.evaluate()

{'eval_loss': 0.515018880367279,
 'eval_runtime': 52.7074,
 'eval_samples_per_second': 18.973,
 'eval_steps_per_second': 1.195,
 'epoch': 3.0}

In [79]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 추론 예시
# 학술논문(0)/공시정보(1)/뉴스(2)/규정(3)/보고서(4)

# 학술논문(0)
text = "국가별 대미 금리차의 요인분해"
text = "실제로 이런 점을 고려하여 5대 재벌과 6～30대 재벌에 대 해 따로 더미변수를 넣어보았지만, 추정 결과에서는 둘 간에 유의한 차이가 나타나지 않았다."
# 공시정보(1)
text = "상기 '2.사채의 권면(전자등록)총액'은 5년 조기 중도상환옵션 증권 2,090억 원, 10년 조기 중도상환옵션 증권 600억 원입니다."
text = "주식회사 창대핫멜시트(이하""당사"")는 2003년 12월 개인사업자로 창업하여 2009년 4월에 법인으로 전환하였습니다."
# 뉴스(2)
text = "NH농협은행 광주본부는 8일 광주시교육청, 초록우산어린이재단 광주본부에 돌봄 이웃 청소년을 위한 코로나19 감염 예방 물품(건강꾸러미)을 전달했다."
text = "코어로직은 40여년간의 주택 거래 자료 등을 토대로 집값 추세를 추적하는 코어로직 HPI지수를 산출하고 있으며 매달 보고서를 통해 예측치도 공개하고 있다."
# 규정(3) -
text = "제1항의 규정에 의하여 협상을 의뢰한 경우 주무부서의 장은 관계부처협의 등 협상에 필요한 사항을 지원하여야 하며, 당해 사업을 담당하는 과장 또는 주무부서의 장이 임명한 자는 협상단원으로 참가하여야 한다."
text = "(나) 임대기간 : 총 50년 범위내(10년 단위로 갱신계약할 수 있음)"
# 보고서(4) -
text = "설립목적에 규정되어 있지 않더라도 지급결제의 원활화와 금융시장의 안정 유지도 실질적으로 한국은행의 중요한 정책목표라고 할 수 있다."
text = "주민세는 단독 세대수 증가, 법인･개인사업자 증가 추이와 30세 미만 미혼 세대주 과세제외 추진에 따른 감소분을 감안하여 1.1% 반영"

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model(**inputs)
predicted_class = outputs.logits.argmax(-1).item()
print("예측 클래스:", predicted_class)

예측 클래스: 4
